In [ ]:
#----import libraries----------------------------------------#
import gym
import numpy as np
from gym import wrappers
import random as rand
#-----set up environment------------------------------------#
env = gym.make('CartPole-v0')

# Random weight generation method

In [ ]:
best_moves = 0
episode_length = []
best_weights = np.zeros(4) #emmpty 4 digit array

for i in range (100):
    new_weights = [rand.uniform(0,1),rand.uniform(0,1),rand.uniform(0,1),rand.uniform(0,1)]
    moves_length = []
    for j in range(100):
        observation = env.reset()
        done = False
        count = 0
        while not done:
            count+=1
            action = 1 if np.dot(observation,new_weights)>0 else  0
            observation,reward,done,_ = env.step(action)
            if done:
                break
        moves_length.append(count)
    avg_moves = float(sum(moves_length)/len(moves_length))
    if avg_moves > best_moves:
        best_moves = avg_moves
        best_weights = new_weights
    episode_length.append(avg_moves)
    if i % 20 == 0:
        print("best moves is ", best_moves)
done = False
counr = 0
env = wrappers.Monitor(env, 'movefiles',force=True) #save files

# Q Learning method

In [ ]:
#converting a continuouss state into a discrete state
MAX_STATES = 10**5
GAMMA = 0.7   #discount factor
ALPHA = 0.01 #Learning rate

#q table disctionary lookup(like weights) gets state with highest reward?
def q_states_dict(q):
    max_q = float('-inf')
    for key,val in q.items():
        if val > max_q:
            max_q = val
            max_key = key
    return max_key,max_q

# our inputs
def create_bins():
    bins = np.zeros((4,10))
    bins[0] = np.linspace(-4.8,4.8,10)
    bins[1] = np.linspace(-10,10,10)
    bins[2] = np.linspace(-41.8,41.8,10)
    bins[3] = np.linspace(-10,10,10)
    return bins
#digitize bins since we are converting a continuous state to a discrete state

def assign_bins(observation,bins):
    env_state = np.zeros(4) #creates a single array to store the 4 states
    for i in range(4):
        env_state[i] = np.digitize(observation[i],bins[i])
    return env_state

def get_state_as_string(env_state):
    env_string = ''.join(str(int(e)) for e in env_state)
    return env_string

def get_all_states_string():
    env_states = []
    for i in range(MAX_STATES):
        env_states.append(str(i).zfill(4))
    return env_states

def initialise_q():
    Q = {}
    all_states = get_all_states_string()
    for state in all_states:
        Q[state] = {}
        for action in range(env.action_space.n):
            Q[state][action] = 0 #could be anything?....like a bias?...
    return Q

def play_a_game(bins,Q,eps=0.2):
    observation = env.reset()
    done = False
    count = 0
    state = get_state_as_string(assign_bins(observation,bins))
    total_reward = 0
    while not done:
        count+=1
        if rand.uniform(0,1) > eps:
            act = env.action_space.sample() #pick a random action
        else:
            act = q_states_dict(Q[state])[0]
        observation,reward,done,_ = env.step(act)
        total_reward+=reward
        if done and count < 250:
            reward = -400
        new_state = get_state_as_string(assign_bins(observation,bins))
        al,max_q_slal = q_states_dict(Q[new_state])
        Q[state][act]+=ALPHA*(reward + GAMMA*max_q_slal - Q[state][act])
        state,act = new_state, al
    return total_reward, count

def play_multiple(bins, N = 100000):
    Q = initialise_q()
    
    count=[]
    reward = []
    for n in range(N):
        epsilon = rand.uniform(0,1)
        ep_reward,ep_length = play_a_game(bins,Q,epsilon)
        if n % 200 == 0:
            print("epsilon: ",epsilon, "Reward: ", ep_reward)
        count.append(ep_length)
        reward.append(ep_reward)
    return count,reward

In [ ]:
bins = create_bins()
x,y = play_multiple(bins)